# import_fmi_geotiff

In [53]:
from pysteps.io.importers import import_fmi_geotiff
from pyprojroot import here
import numpy as np

home = here()

field,_, metadata = import_fmi_geotiff(str(home/"../data/202201010010_FIN-DBZ-3067-250M.tif"))

print(field.shape)
print(np.nanmin(field), np.nanmax(field))
metadata

(7168, 5120)
-32.0 56.0


{'projection': '+proj=utm +zone=35 +ellps=GRS80 +units=m +no_defs',
 'x1': -219875.0,
 'y1': 6257875.0,
 'x2': 1060125.0,
 'y2': 8049875.0,
 'xpixelsize': 250.0,
 'ypixelsize': 250.0,
 'yorigin': 'upper',
 'institution': 'Finnish Meteorological Institute',
 'unit': 'dBZ',
 'transform': 'dB',
 'accutime': 5.0,
 'threshold': np.float64(-31.0),
 'zerovalue': np.float64(-32.0),
 'cartesian_unit': 'm',
 'zr_a': 223.0,
 'zr_b': 1.53}

In [54]:
field

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], shape=(7168, 5120))

# 数据校验

In [46]:
from pyprojroot import here
import pandas as pd
import os

home = here()

filelist = os.listdir(home / "../data")
filenames = pd.DataFrame(filelist,columns=["filename"])
filenames['datetime'] = filenames['filename'].str.split('_').str[0]
filenames['datetime'] = pd.to_datetime(filenames['datetime'])
filenames['datetime_txt'] = filenames['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
filenames = filenames.set_index('datetime').sort_index()
filenames = filenames[~filenames.index.duplicated(keep='first')]

full_range = pd.date_range(start=filenames.index.min(), end=filenames.index.max(), freq='15min')
missing_times = full_range.difference(filenames.index)
if len(missing_times) > 0:
    print(f"发现 {len(missing_times)} 个缺失的时间点:")
    print(missing_times)
else:
    print("数据连续，无缺失时间点。")


发现 3 个缺失的时间点:
DatetimeIndex(['2022-01-01 23:55:00', '2022-01-02 23:55:00',
               '2022-01-03 23:55:00'],
              dtype='datetime64[us]', freq=None)


In [51]:
filenames.iloc[-50:][['datetime_txt']].to_csv('test.txt', index=False)
filenames.iloc[-100:-50][['datetime_txt']].to_csv('val.txt', index=False)
filenames.iloc[:-100][['datetime_txt']].to_csv('train.txt', index=False)

# datamodules

In [ ]:
from datamodules import Rainnet_FMICompositeDataModule
from utils import load_config, setup_logging
from pyprojroot import here

home = here()

confpath = home / "../config/example"
dsconf = load_config(confpath / "FMIComposite.yaml")
outputconf = load_config(confpath / "output.yaml")
modelconf = load_config(confpath / "rainnet.yaml")

# setup_logging(outputconf.logging)

datamodel = Rainnet_FMICompositeDataModule(dsconf, modelconf.train_params)
datamodel.setup("predict")
predict_dataloader = datamodel.predict_dataloader()

# iter(predict_dataloader).next()
for batch in predict_dataloader:
    print(batch)
    break

In [ ]:
# 